In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
from sklearn.preprocessing import StandardScaler
import pubchempy as pcp
import matplotlib.pyplot as plt
import os


import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import Data

In [ ]:
cek = pd.read_csv("../data/string_interactions_short.tsv default node.csv")
cek

In [ ]:
def eigen(i):
    df=pd.read_csv('../data/'+i+'.csv')
    df_name=df['name']
    df=df[['BetweennessCentrality','ClosenessCentrality','ClusteringCoefficient','Degree','Radiality','Stress','TopologicalCoefficient']]#
    scaler = StandardScaler()
    df_scaled=(scaler.fit_transform(df))
    print(df_scaled)
    n_samples = df_scaled.shape[0]
    print(n_samples)
    pca = PCA()
    X_transformed = pca.fit_transform(df_scaled)
    print(X_transformed)
    X_centered = df_scaled - np.mean(df_scaled, axis=0)
    cov_matrix = np.dot(X_centered.T, X_centered) / n_samples
    eigenvalues = pca.explained_variance_
    print('clus ',i)
    print('var ratio:',pca.explained_variance_ratio_[0])
    print('eigen value:',eigenvalues[0])
    print('eigen vector:',pca.components_[0])
    print('_______________________________________________')

    df_scaled=pd.DataFrame(df_scaled,columns=['BetweennessCentrality','ClosenessCentrality','ClusteringCoefficient','Degree','Radiality','Stress','TopologicalCoefficient'])
    df_scaled['Overall']=0.0
    for k in range(0,n_samples):
        temp=0
        for j in range(0,len(pca.components_[0])):
            temp=temp+pca.components_[0][j]*df_scaled[df_scaled.columns[j]][k]
        df_scaled['Overall'][k]=temp
        print('overall ',temp)
    pd.concat([df_name, df_scaled], axis=1, sort=False).to_excel('../data/centrality_'+str(i)+'.xlsx')

In [ ]:
eigen('string_interactions_short.tsv default node')

In [ ]:
data = pd.read_excel('../data/centrality_string_interactions_short.tsv default node.xlsx')
data

In [ ]:
x=data.iloc[:,1]
x

In [ ]:
y=data.iloc[:,9]
y

In [ ]:
plt.figure(figsize=(8,20))
plt.barh(x, y, color='lightcoral')

plt.ylabel('Skor', size=14)
plt.xticks(size=10)
plt.yticks(size=10)

plt.show()

In [ ]:
Pakai = data[data['Overall'] >= 0].sort_values(by='Overall',ascending = False)
# Pakai.to_excel('Data PKM Alzheimer/Centrality_Overall_More_Than_Zero.xlsx')
Pakai

In [ ]:
Pakai = Pakai.reset_index(drop = True).drop(columns = 'Unnamed: 0')

In [ ]:
Nama = Pakai['name']
Nama

In [ ]:
# smiles_dict = {}  # Empty dictionary to store key-value pairs
# for i in Nama :
#     results = pcp.get_compounds(i, 'name')
#     for compound in results:
#         Smiles = compound.isomeric_smiles
#         smiles_dict[i] = Smiles

In [ ]:
# keys = smiles_dict.keys()

In [ ]:
# results = pcp.get_compounds('TNF', 'name')
# for compound in results:
#     print(compound.isomeric_smiles)
#     print(compound.iupac_name)
#     print(compound.synonyms)

In [ ]:
best_protein_candidate = Pakai.head(10).copy()
best_protein_candidate

In [ ]:
best_protein_candidate["name"].to_list()

In [ ]:
directory_path = '../data/compounds'

csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

compounds_df = []
for csv_file in csv_files:
  csv_path = os.path.join(directory_path, csv_file)
  df = pd.read_csv(csv_path)
  compounds_df.append(df)

combined_compounds_df = pd.concat(compounds_df, ignore_index=True)
combined_compounds_df.head()

In [ ]:
combined_compounds_df.columns

In [ ]:
compounds_compact_summary_df = combined_compounds_df[['cid', 'cmpdname', 'isosmiles', 'canonicalsmiles']]
compounds_compact_summary_df.head(10)

In [ ]:
compounds_compact_summary_df.to_csv('../data/results/compounds_summary_results.csv')

# Generate Ligand Fingerprint

In [ ]:
isosmiles_list = compounds_compact_summary_df['isosmiles'].to_list()
print(isosmiles_list[0])

compound = pcp.get_compounds(isosmiles_list[0], 'smiles', record_type='3d')[0]
fingerprint = compound.to_dict(properties=['fingerprint'])['fingerprint']
print("PubChem Fingerprint:", fingerprint)